In [1]:
import numpy as np
from scipy.special import softmax
import matplotlib.pyplot as plt
from copy import deepcopy

def rs(state, action, a, b, c, d, r, l):
    stp = rt = None
    if state == a and action == r:
        stp, rt = c, 0
    elif state == a and action == l:
        stp, rt = b, 0
    elif state == b:
        stp, rt = d, rv + np.random.randn()
    return stp, rt


def rbar(state, action, a, b, r, l, rv):
    if state == a and (action == r or action == l):
        return 0
    elif state == b:
        return rv
    else:
        return 0

def update(state, action, Q, a, b, c, d, r, l):
    maxQ = np.zeros(4)
    for s in range(4):
        maxQ[s] = np.max(Q[s])
    if state == a and action == r:
        return maxQ[c]
    elif state == a and action == l:
        return maxQ[b]
    elif state == b:
        return maxQ[d]
    else:
        return 0


def zeroQ(Q):
    for i in range(4):
        Q[i].fill(0.0)

def alphat(alpha0, t):
    return alpha0 / (1 + 0.001 * t)

def softprob(x):
    x = x - np.max(x)
    x = np.exp(x)
    return x / np.sum(x)

def argmaxR(x):
    p = softmax(1000 * x)
    return np.random.choice(np.arange(len(x)), p=p)

# define states and actions
a, b, c, d = 0, 1, 2, 3
l, r = 0, 1

rv = -0.1
gamma = 0.99
alpha0 = 0.1
eps = 0.1
Exps = 1000
Episodes = 2000

# Initialization
Q = [np.zeros((2,)), np.zeros((8,)), np.zeros((1,)), np.zeros((1,))]
zeroQ(Q)

QA = [np.copy(q) for q in Q]
QB = [np.copy(q) for q in Q]

# Get the true solution to the MDP
Qnew = deepcopy(Q)
zeroQ(Qnew)

for _ in range(1000):
    for state in [a, b, c, d]:
        for action in range(len(Q[state])):
            Qnew[state][action] = rbar(state, action, a, b, r, l, rv) + gamma * update(state, action, Q, a, b, c, d, r, l)
    Q = deepcopy(Qnew)

QmaxTrue = Q[a][l]

# Initialize other necessary variables
Qend = np.zeros((Exps, Episodes))
dQend = np.zeros((Exps, Episodes))
sQend = np.zeros((Exps, Episodes))
cQend = np.zeros((Exps, Episodes))
bQend = np.zeros((Exps, Episodes))

In [4]:
# Initialization
states = ['A', 'B', 'C', 'D']
actions = {
    'A': ['Left', 'Right'],
    'B': ['Action1', 'Action2', 'Action3', 'Action4', 'Action5', 'Action6', 'Action7', 'Action8']
}

# Correctly initialize observed_counts only for states with defined actions
observed_counts = {
    state: {action: {'observed_successes': 0, 'observed_failures': 0} for action in actions.get(state, [])}
    for state in states if state in actions
}


def update_observed_counts(state, action, reward, observed_counts):
    """
    Update the observed successes or failures based on the actual reward.
    """
    if reward > 0:
        observed_counts[state][action]['observed_successes'] += 1
    else:
        observed_counts[state][action]['observed_failures'] += 1


def calculate_action_probs(state, observed_counts, actions):
    """
    Calculate action probabilities based on observed successes and failures for the Bayesian approach.
    """
    action_probs = np.zeros(len(actions[state]))
    for action_index, action in enumerate(actions[state]):
        observed = observed_counts[state][action]
        total_successes = observed['observed_successes']
        total_failures = observed['observed_failures']
        # Bayesian probability calculation
        action_prob = (total_successes + 1) / (total_successes + total_failures + 2)  # Adding pseudo-counts
        action_probs[action_index] = action_prob
    return action_probs


epsilon = 0.1
# Implementing the Smoothed Q-Learning algorithm
actionsBayes = []
for experiment in range(Exps):
    Q = np.zeros((len(states), max(len(actions[s]) for s in actions)))  # Reset Q values for each experiment
    ct = [0]  # Counter for learning rate update
    actionsBayes_ex = []

    for episode in range(Episodes):
        st = 'A'  # Starting state
        while True:
            if np.random.rand() < epsilon:
                at = np.random.choice(actions[st])
            else:
                action_probs = calculate_action_probs(st, observed_counts, actions)
                at = actions[st][np.argmax(action_probs)]

            at_index = actions[st].index(at)
            st_index = states.index(st)
            stp, rt = rs(st_index, at_index, 0, 1, 2, 3, 0, 1)  # Assumed mapping of action indexes
            update_observed_counts(st, at, rt, observed_counts)
            ct[0] += 1
            alpha = alphat(alpha0, ct[0])

            # Correct the Q-value update
            if stp not in [2, 3]:  # Check if next state is not terminal
                next_state_actions = actions[states[stp]]
                next_action_probs = calculate_action_probs(states[stp], observed_counts, actions)
                expected_future_reward = np.sum(next_action_probs * Q[stp, :len(next_state_actions)])
                Q[st_index][at_index] += alpha * (rt + gamma * expected_future_reward - Q[st_index][at_index])
            else:
                Q[st_index][at_index] += alpha * (rt - Q[st_index][at_index])  # Terminal state

            if stp in [2, 3]:  # Terminal states 'C' or 'D'
                break
            st = states[stp]  # Update state for next iteration

        actionsBayes_ex.append(at_index)
    actionsBayes.append(actionsBayes_ex)

In [5]:
print(actionsBayes)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
# avg_abs_error_Q = np.mean(np.abs(Qend - QmaxTrue), axis=0)
# avg_abs_error_dQ = np.mean(np.abs(dQend - QmaxTrue), axis=0)
# avg_abs_error_sQ = np.mean(np.abs(sQend - QmaxTrue), axis=0)
# avg_abs_error_cQ = np.mean(np.abs(cQend - QmaxTrue), axis=0)
avg_abs_error_bQ = np.mean(np.abs(bQend - QmaxTrue), axis=0)

# min_length = min(
#     # min(len(sublist) for sublist in actionsQ),
#                  # min(len(sublist) for sublist in actionsdQ),
#                  # min(len(sublist) for sublist in actionssQ),
#                  # min(len(sublist) for sublist in actionscQ),
#                  min(len(sublist) for sublist in actionsBayes))

min_length = min(min(len(sublist) for sublist in actionsBayes))
# print(min_length)
# actionsQ_trimmed = [sublist[:min_length] for sublist in actionsQ]
# actionsdQ_trimmed = [sublist[:min_length] for sublist in actionsdQ]
# actionssQ_trimmed = [sublist[:min_length] for sublist in actionssQ]
# actionscQ_trimmed = [sublist[:min_length] for sublist in actionscQ]
actionsbQ_trimmed = [sublist[:min_length] for sublist in actionsBayes]
# print(actionsQ_trimmed)
# avg_action_Q = np.mean([[action == l for action in sublist] for sublist in actionsQ_trimmed], axis=0)
# avg_action_dQ = np.mean([[action == l for action in sublist] for sublist in actionsdQ_trimmed], axis=0)
# avg_action_sQ = np.mean([[action == l for action in sublist] for sublist in actionssQ_trimmed], axis=0)
# avg_action_cQ = np.mean([[action == l for action in sublist] for sublist in actionscQ_trimmed], axis=0)
avg_action_bQ = np.mean([[action == l for action in sublist] for sublist in actionsbQ_trimmed], axis=0)

# print(avg_action_Q)
# print(avg_action_dQ)
# print(avg_abs_error_bQ)
# Create subplots
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

# Plot the average absolute errors for each method
# axs[0].plot(avg_abs_error_Q, label='Q learning')
# axs[0].plot(avg_abs_error_dQ, label='Double Q learning')
# axs[0].plot(avg_abs_error_sQ, label='Smoothed (softmax) Q learning')
# axs[0].plot(avg_abs_error_cQ, label='Smoothed (clipmax) Q learning')
axs[0].plot(avg_abs_error_bQ, label='Smoothed (Bayes) Q learning')
axs[0].legend()
axs[0].set_xlabel('Episodes')
axs[0].set_ylabel('Average Absolute Error')
axs[0].set_title('Comparison of Q Learning Methods')
axs[0].grid(True)

# Plot the average actions for each method
# axs[1].plot(avg_action_Q, label='Q learning')
# axs[1].plot(avg_action_dQ, label='Double Q learning')
# axs[1].plot(avg_action_sQ, label='Smoothed (softmax) Q learning')
# axs[1].plot(avg_action_cQ, label='Smoothed (clipmax) Q learning')
axs[1].plot(avg_action_bQ, label='Smoothed (Bayes) Q learning')
axs[1].legend()
axs[1].set_xlabel('Episodes')
axs[1].set_ylabel('Average Actions')
axs[1].set_title('Comparison of Actions in Q Learning Methods')
axs[1].grid(True)
# Display the figure
plt.show()

TypeError: 'int' object is not iterable